In [31]:
from pathlib import Path

def hexToBytes(s):
    bs = []
    for i in range(0, len(s), 2):
        scod = s[i: i + 2]
        bs.append(int(scod, 16))
    return  bytes(bs)

# Зашифрований текст, перетворений на байтові рядки
txt1 = Path("shifrovka.txt").read_text()
lines = [hexToBytes(s.strip()) for s in txt1.splitlines() if s.strip()]
# lines = dict(enumerate(lines))
print(len(lines))

txt2 = Path("en_words.txt").read_text()
words = [bytes(s.strip(), encoding='utf8') for s in txt2.splitlines() if s.strip()]
words = sorted(words, key=len, reverse=True) 
print(len(words))

19
1013


In [32]:
import re, sys, os
from pprint import pprint
from operator import xor
from collections import defaultdict, Counter

patterns = [
    (b' %s ', bytes),
    (b' %s. ', bytes),
    (b' %s,', bytes),
    (b' %s.', bytes),
    (b' %s, ', bytes),
    (b'. %s ', bytes),
    (b'. %s ', bytes.title),
    (b', %s ', bytes),
    (b'.%s ', bytes),
    (b'.%s ', bytes.title),
    (b',%s ', bytes),
]

def test_rez(s0):
    ss = s0.split()  
    for s in ss:
        m = re.match(rb'^([A-Z]?[a-z]+-?[a-z]*)|(\d+[-.,]?\d*)[.,]?$', s)
        if not m:
            return
    return True

In [33]:
# сортовані по спаду довжині рядка
lines2 = sorted(lines, reverse=True, key=len)
# несортовані, порядок як у початковій шифровці
# lines2 = lines

In [34]:
# Якщо кожний рядок зашифрований спільним ключем (послідовністю байтів),
# спробуємо знайти можливі значення і їх статистику для кожної позиції
rez_codes = {} # словник статистики {позиція: {код: кількість, ...}, ...}
for word0 in words:
    for ps in patterns:
        word = ps[1](ps[0] % word0)
        # print(word)
        # зсувом слова word для XOR кожної пари рядків шукаємо правдоподібний текст
        # якщо він правдоподібний для попередніх і наступних рядків зашифрованого тексту, то скоріш за все
        # слово правильне і треба добавити статистику для цих позицій
        # (можна не тільки сусідніх, а в будьяких, тільки це накладніше)
        for L in range(1, len(lines2) - 1):   # L - це номер рядка
            # результат побайтного XOR для поточного і попереднього рядків
            xorL0 = bytes(map(xor, lines2[L], lines2[L-1]))
            # результат побайтного XOR для поточного і наступного рядків
            xorL2 = bytes(map(xor, lines2[L], lines2[L+1]))
            # готовимо позицію зсувів
            for i in range(min(len(xorL0), len(xorL2)) - len(word) + 1):
                # одержуемо результати з зсунутими word-ами (можливо правильні фрагменти іншого рядка)
                rez0 = bytes(map(xor, word, xorL0[i: i + len(word)]))
                rez2 = bytes(map(xor, word, xorL2[i: i + len(word)]))
                # перевіряємо правдоподібність результатів
                if test_rez(rez0) and test_rez(rez2):
                    # результати правдоподібні, будемо одережувати код для цих позицій 
                    # і додаємо словник статистики
                    icodes = bytes(map(xor, word, lines2[L][i:i + len(word)]))
                    for j, cod in enumerate(icodes):
                        if i + j not in rez_codes:
                            rez_codes[i + j] = {}
                        if cod not in rez_codes[i + j]:
                            rez_codes[i + j][cod] = 0
                        rez_codes[i + j][cod] += 1
# Залишаємо для декодування тільки ті коди позицій, у яких найбільше значення
rez_codes1 = {}
for p, codes in rez_codes.items():
    codes = sorted(codes.items(), key=lambda x: x[1])
    rez_codes1[p] = codes[-1][0]  # код словника з найбільшим значенням

def_code = 0x42
print('''\
позиції 001111111111222222222233333333334444444444
01234567890123456789012345678901234567890123456789''')
for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = b ^ rez_codes1.get(i, def_code)
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))

In [ ]:
from copy import deepcopy
from functools import reduce 

rez2_codes = deepcopy(rez_codes1)

# Позиції, яких необхідно замінити символи
manuals = {
  5: b'oh', 
 20: b"dh", 
  0: b'ma', 
  1: b'Nn', 
 40: b'n/', 
 41: b't8', 
 42: b'u9', 
 43: b'ma', 
 44: b'e ', 
 45: b'l ', 
 46: b'y ', 
 47: b',k', 
}
# Зміна кодів для заміни символів в позиціях
for p, x in manuals.items():
    rez2_codes[p] = rez2_codes.get(p, def_code) ^ x[0] ^ x[1]
   
for line in lines:
    rez = []
    for i, b in enumerate(line):
        ch = b ^ rez2_codes[i]#.get(i, def_code)
        rez.append(ch)
    print(bytes(rez).decode(encoding='utf8', errors='replace'))

for who would bear the whips and scorns of timen
th'oppressor's wrong, the proud man's contumelyn
the pangs of dispriz'd love, the law's delay,
the insolence of office, and the spurns
that patient merit of th'unworthy takes,
when he himself might his quietus make
with a bare bodkin? Who would fardels bear,
to grunt and sweat under a weary life,
but that the dread of something after death,
the undiscovere'd country, from whose bourn
no traveller returns, puzzles the will,
and makes us rather bear those ills we have
than fly to others that we know not of?
thus conscience doth make cowards of us all,
and thus the native hue of resolution
is sicklied o'er with the pale cast of thought,
and enterprises of great pith and moment
with this regard their currents turn awry
and lose the name of action.
